import lib

In [131]:
import time
import requests
from bs4 import BeautifulSoup as bs
import re
from datetime import date
import pandas as pd
import json
from pathlib import Path 

data mold

In [132]:
result = {'names': [], 'email': [], 'address': [], 'phone': [], 'clinic name': []}

get data

In [133]:
def getLinks(html_doc):
    linkRecords = []
    soup =  bs(html_doc, 'html.parser')
    resultsFind = soup.find_all("tbody", {"class": "fap-desktop-view"})
    for resultFind in resultsFind:
        for a in resultFind.find_all('a', href=True):
            linkRecords.append('https://www.psych.on.ca/Public/' + a['href'])
    return linkRecords
    

In [134]:
def decodeCFEmail(fp):
    try:
        r = int(fp[:2],16)
        email = ''.join([chr(int(fp[i:i+2], 16) ^ r) for i in range(2, len(fp), 2)])
        return email
    except (ValueError):
        pass

In [135]:
def getData(html_doc):
    data = {'names': None, 'email': None, 'address': None, 'phone': None, 'clinic name': None}
    soup =  bs(html_doc, 'html.parser')
    try:
        data['names'] = soup.find('p', {'class': 'fap-details-title'}).getText()
        infos = soup.find_all('p', {'class': 'info'})
        cfemail = infos[0].find('span', {'class': '__cf_email__'})
        data['email'] = decodeCFEmail(cfemail['data-cfemail'])
        data['address'] = infos[2].getText().replace('Address: ', '') #address
        data['phone'] = infos[3].getText().replace('Phone: ', '')
        return data
    except:
        pass


In [136]:
indexPageEnd = 80

payload={}
headers = {
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
  'Cookie': 'ARRAffinity=8f34f6436376f83ac80fcd318a5e17380aa993a73269f934accc27d66e80d9a9; ARRAffinitySameSite=8f34f6436376f83ac80fcd318a5e17380aa993a73269f934accc27d66e80d9a9; ASP.NET_SessionId=zrcwy4w1c533jv0pqq4lyoda'
}

for page in range(1,indexPageEnd):
  url = "https://www.psych.on.ca/Public/Find-a-Psychologist?page=" + str(page)
  response = requests.request("GET", url, headers=headers, data=payload)
  clinicLinks = getLinks(response.text)
  for link in clinicLinks:
    payload={}
    headers = {
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
      'Cookie': 'ARRAffinity=8f34f6436376f83ac80fcd318a5e17380aa993a73269f934accc27d66e80d9a9; ARRAffinitySameSite=8f34f6436376f83ac80fcd318a5e17380aa993a73269f934accc27d66e80d9a9; ASP.NET_SessionId=zrcwy4w1c533jv0pqq4lyoda'
    }
    subResponse = requests.request("GET", link, headers=headers, data=payload)
    print(link)
    data = getData(subResponse.text)
    if data:
      result['names'].append(data['names'])
      result['email'].append(data['email'])
      result['address'].append(data['address'])
      result['phone'].append(data['phone'])
      result['clinic name'].append(data['clinic name'])



https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2630
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2635
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2636
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2637
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2640
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2641
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2650
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2651
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2651
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2653
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2659
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2663
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2669
https://www.psych.on.ca/Public/Find-a-Psychologist-Details?psyID=2671
https://www.psych.on

extract to csv file

In [137]:
df = pd.DataFrame(result)  
with pd.ExcelWriter("./outClinic.xlsx") as writer:
    df.to_excel(writer, sheet_name="Sheet1") 